In [1]:
import tkinter as tk
from tkinter import ttk
from bson import ObjectId
from pymongo import MongoClient

cliente=MongoClient("mongodb://127.0.0.1:27017")
db=cliente.get_database("casting")
modelo={
  "_id":"",
  "nombre": "",
  "apellido": "",
  "edad": 0,
  "color_ojos": "",
  "color_pelo": "",
  "estatura_cm": 0,
  "genero": "",
  "descripcion": "",
  "disponibilidad": ""
}

class FormFactura:
    def __init__(self,main):
        self.root= tk.Toplevel(master=main)
        self.root.title="Mantenimiento Personas"
        self.root.geometry("600x480")
        ##listview para maestro detalle
        self.listview=ttk.Treeview(self.root,columns=["IdProducto","Nombre","Unidades","Precio"])
        self.listview.heading("IdProducto",text="IdProducto")
        self.listview.heading("Nombre",text="Nombre")
        self.listview.heading("Unidades",text="Unidades")
        self.listview.heading("Precio",text="Precio")
        self.listview.grid(column=0,row=0,padx=10,pady=10)

        ##llenar listview
        cliente=MongoClient("mongodb://127.0.0.1:27017")
        db=cliente.get_database("factura_sample")
        doc=db.facturas.find()
        for factura in doc:
            nivel=self.listview.insert("",0,text=factura["numero_factura"])
            for item in factura['items']:
                self.listview.insert(nivel,0,values=[item['producto_id'],item['nombre_producto'],item['cantidad'],item['precio_unitario']])

class FormMain:
    def __init__(self):
        self.root=tk.Tk()
        #creo la barra de menús
        menu_bar=tk.Menu(self.root)
        #primer menú de la barra
        menu_mantenimiento=tk.Menu(menu_bar,title="Mantenimiento",tearoff=0)
        #Submenus de mantenimiento
        menu_mantenimiento.add_cascade(label="Mantenimiento Clientes",command=self.mostrar_frm_clientes)
        menu_mantenimiento.add_cascade(label="Facturas",command=lambda:FormFactura(self.root))
        #agrego al menu_principal el menu de mantenimiento
        menu_bar.add_cascade(menu=menu_mantenimiento,label="Mantenimiento")
        #agrego la barra de menus a la ventana
        self.root.configure(menu=menu_bar)
    def visualizar(self):
        self.root.mainloop()
    
    def mostrar_frm_clientes(self):
        FormCRUD(self.root)

class FormCRUD:
    def __init__(self,main):
#Creación de la ventana principal.
        self.root= tk.Toplevel(master=main)
        self.root.title="Mantenimiento Personas"
        self.root.geometry("600x480")
#Creación del panel principal        
        self.frame= tk.Frame(self.root,background="#0F8A86")
        self.frame.pack(expand=True,fill="both")

#Variables asociadas a las Entry
        self.txt_id=tk.StringVar()
        self.txt_nombre=tk.StringVar()
        self.txt_apellido=tk.StringVar()
        self.txt_edad=tk.IntVar()
        self.txt_color_ojos=tk.StringVar()
        self.txt_color_pelo=tk.StringVar()
        self.txt_estatura_cm=tk.IntVar()
        self.txt_genero=tk.StringVar()
        self.txt_descripcion=tk.StringVar()

        lista_variables=[self.txt_id,self.txt_nombre,self.txt_apellido,self.txt_edad,self.txt_color_ojos,self.txt_color_pelo,
                         self.txt_estatura_cm,self.txt_genero,self.txt_descripcion]
#Creación de las etiquetas
        
        i=0
        for label,txtvar in zip(modelo.keys(),lista_variables):
            tk.Label(self.frame,background="#0F8A86",foreground="Yellow",text=label).grid(row=i,column=0,sticky="w",padx=(10,10),pady=(10,10))
            tk.Entry(self.frame,foreground="blue",textvariable=txtvar).grid(row=i,column=1,sticky="w",padx=(10,10),pady=(10,10))
            i+=1
#Creación del botón de búsqueda:
#        self.Buscar=tk.Button(self.frame,command=lambda:self.onBuscar(self.txt_id.get())).grid(row=0,column=2,sticky="w",padx=(10,10),pady=(10,10))
        self.Buscar=tk.Button(self.frame,text="Buscar",command=self.onBuscar).grid(row=0,column=2,sticky="w",padx=(10,10),pady=(10,10))
#Resto de botones
        self.Alta=tk.Button(self.frame,text="Alta",width=15,command=self.onAlta)
        self.Alta.grid(row=i,column=0,sticky="w",padx=(3,3),pady=(10,10))
        self.Baja=tk.Button(self.frame,text="Baja",width=15,command=self.onBaja)
        self.Baja.grid(row=i,column=1,sticky="w",padx=(3,3),pady=(10,10))
        self.Modificacion=tk.Button(self.frame,width=15,text="Modificación",command=self.onModificacion)
        self.Modificacion.grid(row=i,column=2,sticky="w",padx=(3,3),pady=(10,10))
        
    #def visualizar(self):
    #    self.root.mainloop()
    
    def onBuscar(self):
        print(self.txt_id.get())
        doc=db.Personas.find_one({"_id":ObjectId(self.txt_id.get())}) 
        if doc is None:
            print("Elemento no encontrado")
            self.Alta.config(state=tk.ACTIVE)
            self.Baja.config(state=tk.DISABLED)
            self.Modificacion.config(state=tk.DISABLED)
        else:
            self.txt_nombre.set(doc["nombre"])
            self.txt_apellido.set(doc["apellido"])
            self.txt_color_ojos.set(doc["color_ojos"])
            self.txt_color_pelo.set(doc["color_pelo"])
            self.txt_descripcion.set(doc["descripcion"])
            self.txt_edad.set(doc["edad"])
            self.txt_estatura_cm.set(doc["estatura_cm"])
            self.txt_genero.set(doc["genero"])
            self.Alta.config(state=tk.DISABLED)
            self.Baja.config(state=tk.ACTIVE)
            self.Modificacion.config(state=tk.ACTIVE)
    
    def onAlta(self):
        modelo.update({"_id":ObjectId(self.txt_id.get())})   
        modelo.update({"nombre":self.txt_nombre.get()})
        modelo.update({"apellido":self.txt_apellido.get()})
        modelo.update({"edad":self.txt_edad.get()})
        modelo.update({"color_ojos":self.txt_color_ojos.get()})
        modelo.update({"color_pelo":self.txt_color_pelo.get()})
        modelo.update({"genero":self.txt_genero.get()})
        modelo.update({"estatura_cm":self.txt_estatura_cm.get()})
        modelo.update({"descripcion":self.txt_descripcion.get()})
        db.Personas.insert_one(modelo)
    def onBaja(self):
        print(f"Baja {self.txt_id.get()}")    

    def onModificacion(self):
        print(f"Modificación {self.txt_id.get()}")    

In [2]:
app=FormMain()
app.visualizar()